In [1]:
import pyarrow.parquet as pq
import pandas as pd


In some of the original parquet files provided by the city of new york, the dropoff time and pickup time were put in a special format that parquet couldn't read. I changed the format in pyarrow and then read them into parquet and saved them.

In [2]:
parquet_file_path = r"C:\Users\EvanSpiller\Downloads\fhv_tripdata_2018-06.parquet"
# Read the Parquet file using PyArrow
table = pq.read_table(parquet_file_path)
datetimecolumn = pd.DataFrame(table['pickup_datetime']) #reading the problem column in a pandas df
datetimecolumn.rename(columns={0: 'pickup_datetime'}, inplace=True) #renaming it
datetimecolumn['pickup_datetime'] = str(datetimecolumn['pickup_datetime']) #turning it into a string
datetime_strings = []
#using a for loop to loop through the strings and create new strings that can be converted into regular datetimes
for x in range(len(datetimecolumn['pickup_datetime'])):
    dummy= datetimecolumn['pickup_datetime'][x]
    string = str.split(dummy)[1]+" " + str.split(dummy)[2] 
    datetime_strings.append(string)
datetimecolumn['datetime_string'] = datetime_strings
datetimecolumn['pickup_datetime'] = pd.to_datetime(datetimecolumn['datetime_string'])

In [54]:
#same code but for dropoff datetimes instead of pickup datetimes
datetimecolumn['dropOff_datetime'] = pd.DataFrame(table['dropOff_datetime'])
datetimecolumn['dropOff_datetime'] = str(datetimecolumn['dropOff_datetime'])
datetime_strings = []
for x in range(len(datetimecolumn['dropOff_datetime'])):
    dummy= datetimecolumn['dropOff_datetime'][x]
    string = str.split(dummy)[1]+" " + str.split(dummy)[2]
    datetime_strings.append(string)
datetimecolumn['datetime_string'] = datetime_strings
datetimecolumn['dropOff_datetime'] = pd.to_datetime(datetimecolumn['datetime_string'])

In [55]:
df = pq.read_table(parquet_file_path, columns=['dispatching_base_num',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'])

In [56]:
df = df.to_pandas()
#sending everything but the problem columns to pandas in the original df

In [57]:
datetimecolumn.drop('datetime_string', axis = 1, inplace = True)

In [58]:
merged_df = pd.merge(df, datetimecolumn, left_index=True, right_index=True)
#merging the dataframe of fixed columns with my original df so i have all columns

In [59]:
new_file_path = r"C:\Users\EvanSpiller\Documents\DS BootCamp\Capstone\data\FHV_Compressed\Special_Uncompressed\fhv_tripdata_fixed_2018-06.parquet"
#sending the final file to a csv

In [60]:
import pyarrow

In [61]:
merged_df.to_parquet(new_file_path, engine='pyarrow')


In [62]:
merged_df.isna().sum()
#just some basic file inspection -- 14.5% na's for an important field -- no way to impute values.

dispatching_base_num             0
PUlocationID               3065436
DOlocationID               1570497
SR_Flag                   16425394
Affiliated_base_number        5486
pickup_datetime                  0
dropOff_datetime                 0
dtype: int64

In [73]:
merged_df.index

RangeIndex(start=0, stop=21137951, step=1)

In [1]:
3065436/21137951

0.14502048945046755